In [ ]:
%matplotlib inline
import flopy
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl
import shutil

from flopy.mf6.utils import Mf6Splitter

## Load the existing model

In [ ]:
# load the simple model setup

name = "sv"
existing_ws = pl.Path("../models/synthetic-valley-vg")
ws = existing_ws.parent / "synthetic-valley-vg-parallel"
if ws.is_dir():
    shutil.rmtree(ws)

In [ ]:
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=existing_ws, write_headers=False
)

In [ ]:
gwf = sim.get_model(name)
nlay, ncpl = gwf.disv.nlay.array, gwf.disv.ncpl.array
shape3d = (nlay, ncpl)

In [ ]:
gwf.remove_package("MVR")

### Split the model

In [ ]:
nr_domains = 1

In [ ]:
mfsplit = Mf6Splitter(sim)

In [ ]:
split_array = mfsplit.optimize_splitting_mask(nparts=nr_domains)

#### Plot the splitting array

In [ ]:
pmv = flopy.plot.PlotMapView(model=gwf, layer=0)
pa = pmv.plot_array(split_array)
pmv.plot_inactive(color_noflow="white")
plt.colorbar(pa, shrink=0.6);

In [ ]:
parallel_sim = mfsplit.split_model(split_array)

### Change the simulation workspace and write the files

In [ ]:
parallel_sim.set_sim_path(ws)
parallel_sim.write_simulation()

In [ ]:
models = [parallel_sim.get_model(mname) for mname in parallel_sim.model_names]
for m in models:
    idomain = m.modelgrid.idomain
    print(f"Nr. of active cells in {m.name}:", np.count_nonzero(idomain))

## Run the model in parallel

In [ ]:
parallel_sim.run_simulation(processors=nr_domains)

## Plot the results

Create a dictionary with the simulated heads for each subdomain

In [ ]:
arrays = {}
for idx, model in enumerate(models):
    times = model.output.head().get_times()
    arrays[idx] = model.output.head().get_data(totim=times[-1])

Reconstruct a single array that can be plotted

In [ ]:
arr = mfsplit.reconstruct_array(arrays)

Plot the head in each layer 

In [ ]:
six_panel_figsize = (17.15 / 2.541, 1.4 * 0.8333 * 17.15 / 2.541)
contour_color = "white"
contour_style = "--"

sv_gwt_contour_dict = {
    "linewidths": 0.75,
    "colors": contour_color,
    "linestyles": contour_style,
}

mosaic = [[0, 1, 2], [3, 4, "."]]

In [ ]:
with flopy.plot.styles.USGSMap():
    fix, axd = plt.subplot_mosaic(
        mosaic,
        figsize=six_panel_figsize,
        layout="constrained",
    )

    for k in range(nlay):
        ax = axd[k]
        ax.set_xlim(gwf.modelgrid.extent[:2])
        ax.set_ylim(gwf.modelgrid.extent[2:])
        ax.set_title(f"Layer {k + 1}")
        mm = flopy.plot.PlotMapView(
            model=gwf,
            ax=ax,
            extent=gwf.modelgrid.extent,
            layer=k,
        )
        mm.plot_array(arr, edgecolor="black", lw=0.25)

        mm.plot_bc(package=gwf.sfr, color="cyan")
        mm.plot_bc(package=gwf.lak)
        mm.plot_bc(package=gwf.wel, color="red")

        cs = mm.contour_array(
            arr,
            masked_values=[0],
            levels=np.arange(1, 5, 0.5),
            **sv_gwt_contour_dict,
        )

        ax.clabel(
            cs,
            inline=True,
            fmt="%2.1f",
            fontsize=8,
            inline_spacing=0.5,
            colors="white",
        )